### **Algoritmos aleatorizados**

Un **algoritmo aleatorizado** (randomized algorithm) es un procedimiento computacional que introduce aleatoriedad para influir en su comportamiento. Estos algoritmos no son deterministas: pueden producir resultados distintos con las mismas entradas, dependiendo de las decisiones tomadas mediante variables aleatorias internas.

#### ¿Por qué usar aleatoriedad?

1. **Eficiencia**: Algunos problemas tienen soluciones exactas costosas, pero versiones aleatorizadas ofrecen respuestas "suficientemente buenas" más rápidamente.
2. **Simplicidad**: La introducción de aleatoriedad permite estructuras algorítmicas más simples para problemas complejos (como el balanceo de árboles en promedio).
3. **Evitar peores casos sistemáticos**: Por ejemplo, el algoritmo de ordenamiento *QuickSort* puede tener peor caso $O(n^2)$ con ciertas entradas, pero si se elige el pivote aleatoriamente, se evita este comportamiento de forma probabilística.

#### Clasificación

1. **Las Vegas**: Garantizan resultados correctos, pero el tiempo de ejecución es aleatorio.
   - Ejemplo: Algoritmo de selección aleatoria para encontrar la mediana.
2. **Monte Carlo**: Garantizan tiempos de ejecución limitados, pero pueden retornar resultados incorrectos con baja probabilidad.
   - Ejemplo: Algoritmo de Rabin-Miller para primalidad.

Ambos tipos se utilizan extensamente en algoritmos distribuidos, estructuras probabilísticas, algoritmos de aproximación, hashing, y criptografía.

#### Hashing aleatorizado

En estructuras como el **Bloom filter**, el hashing aleatorizado cumple un papel esencial. Las funciones hash deben distribuir uniformemente los elementos, incluso en presencia de entradas sesgadas. El uso de una **semilla aleatoria** permite garantizar independencia entre funciones de hash o simularla mediante técnicas como el *double hashing*.

En el código:

```python
h1 = murmurhash3_32(key, seed)
h2 = fnv1_hash32(key)
```

La combinación:

$$
\text{hash}_i(x) = h_1(x) + i \cdot h_2(x) + i^2 \bmod m
$$

...es una forma práctica de obtener múltiples funciones de hash independientes desde dos funciones base, minimizando colisiones y distribuyendo uniformemente los bits activados en el filtro.

Esto garantiza una distribución lo más cercana posible al comportamiento de funciones verdaderamente independientes, crucial para la precisión del filtro.

### Bloom filters

Un **Bloom filter**, propuesto por Burton Howard Bloom en 1970, es una estructura de datos probabilística diseñada para resolver un problema clásico:

> ¿Existe una forma eficiente de verificar si un elemento pertenece a un conjunto muy grande, sin almacenar todos los elementos?

#### Principio de funcionamiento

- Se define un vector de bits de tamaño $m$.
- Se escogen $k$ funciones hash.
- Para cada elemento insertado, se computan sus $k$ hashes y se marcan los bits correspondientes.

**Consulta**: Para verificar si un elemento está presente, se comprueba si los $k$ bits están en 1. Si alguno está en 0, se garantiza que el elemento no fue insertado. Si todos están en 1, puede tratarse de un falso positivo.

#### Propiedades clave

- **No hay falsos negativos**.
- **Hay falsos positivos**, cuya probabilidad se puede controlar ajustando parámetros.
- **Muy eficiente en memoria**, mucho más que mantener una tabla hash.
- **Operaciones básicas en $ O(k) $**, con $ k $ pequeño.


#### Formulación matemática

Para un filtro con:
- $ m $: número de bits,
- $ n $: número esperado de elementos insertados,
- $ k $: número de funciones hash,

La probabilidad de que un bit permanezca 0 después de insertar $ n $ elementos es:

$$
p_0 = \left(1 - \frac{1}{m}\right)^{kn} \approx e^{-kn/m}
$$

La probabilidad de que un bit esté en 1:

$$
p_1 = 1 - e^{-kn/m}
$$

La probabilidad de un falso positivo (todos los $ k $ bits estén en 1 para un elemento no insertado):

$$
P_{fp} = (1 - e^{-kn/m})^k
$$

Esta probabilidad disminuye si aumentamos $ m $ o usamos el número óptimo de hash functions $ k \approx \frac{m}{n} \ln 2 $.



In [2]:
import math
import random
import json

# Función de hash FNV-1 de 32 bits (retorna un entero sin signo)
def fnv1_hash32(key: str) -> int:
    fnv_prime = 0x01000193  # Constante primo FNV
    hash_ = 0x811c9dc5      # Valor inicial offset basis
    for c in key:
        hash_ = (hash_ * fnv_prime) & 0xFFFFFFFF  # Multiplica por el primo y limita a 32 bits
        hash_ ^= ord(c)                            # XOR con el valor ASCII del carácter
    return hash_

# Implementación de MurmurHash3 x86 32 bits
def murmurhash3_32(key: str, seed: int = 0) -> int:
    data = key.encode('utf-8')
    length = len(data)
    nblocks = length // 4

    h1 = seed & 0xFFFFFFFF
    c1 = 0xcc9e2d51
    c2 = 0x1b873593

    # Procesamiento de bloques de 4 bytes
    for block_start in range(0, nblocks * 4, 4):
        # Combina 4 bytes en un entero de 32 bits
        k1 = (data[block_start]
              | (data[block_start+1] << 8)
              | (data[block_start+2] << 16)
              | (data[block_start+3] << 24))
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = ((k1 << 15) | (k1 >> 17)) & 0xFFFFFFFF  # Rotación izquierda de 15 bits
        k1 = (k1 * c2) & 0xFFFFFFFF

        # Mezcla con el hash
        h1 ^= k1
        h1 = ((h1 << 13) | (h1 >> 19)) & 0xFFFFFFFF
        h1 = (h1 * 5 + 0xe6546b64) & 0xFFFFFFFF

    # Procesamiento de la cola (últimos bytes)
    tail_index = nblocks * 4
    tail_size = length & 3
    k1 = 0
    if tail_size == 3:
        k1 ^= data[tail_index + 2] << 16
    if tail_size >= 2:
        k1 ^= data[tail_index + 1] << 8
    if tail_size >= 1:
        k1 ^= data[tail_index]
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = ((k1 << 15) | (k1 >> 17)) & 0xFFFFFFFF
        k1 = (k1 * c2) & 0xFFFFFFFF
        h1 ^= k1

    # Finalización (avalancha de bits)
    h1 ^= length
    h1 &= 0xFFFFFFFF
    h1 ^= (h1 >> 16)
    h1  = (h1 * 0x85ebca6b) & 0xFFFFFFFF
    h1 ^= (h1 >> 13)
    h1  = (h1 * 0xc2b2ae35) & 0xFFFFFFFF
    h1 ^= (h1 >> 16)
    return h1

# Serializa valores de forma determinista (JSON ordenado)
def consistent_stringify(value) -> str:
    return json.dumps(value, sort_keys=True, ensure_ascii=False)

class BloomFilter:
    """
    Implementación de un Bloom filter con FNV-1 y MurmurHash3 para hashing múltiple.
    """
    def __init__(self, max_size: int, max_tolerance: float = 0.01, seed: int = None):
        # Validación de parámetros
        if not isinstance(max_size, int) or max_size <= 0:
            raise TypeError(f"maxSize debe ser un entero positivo, recibido: {max_size}")
        try:
            tol = float(max_tolerance)
        except:
            raise TypeError(f"tolerance debe ser un número en (0,1), recibido: {max_tolerance}")
        if tol <= 0 or tol >= 1:
            raise TypeError(f"tolerance debe cumplir 0 < t < 1, recibido: {max_tolerance}")
        if seed is None:
            seed = random.getrandbits(32)  # Semilla aleatoria si no se provee
        if not isinstance(seed, int):
            raise TypeError(f"seed debe ser un entero, recibido: {seed}")

        self._max_size = max_size
        self._seed = seed

        ln2 = math.log(2)
        # Número de bits: m = -n ln p / (ln 2)^2
        self._num_bits = math.ceil(-max_size * math.log(tol) / (ln2**2))
        # Número de hashes: k = (m/n) ln 2  =>  k = -ln p / ln 2
        self._num_hashes = math.ceil(-math.log(tol) / ln2)

        # Prevención de filtros excesivamente grandes
        if self._num_bits > 1_000_000_000:
            raise MemoryError("Demasiada memoria requerida para el Bloom filter")

        num_bytes = math.ceil(self._num_bits / 8)
        self._bits = bytearray(num_bytes)  # Array de bytes para los bits
        self._size = 0                     # Conteo de inserciones únicas

    def _bit_coords(self, index: int):
        """Devuelve el índice de byte y el desplazamiento de bit para un índice global."""
        byte_idx = index // 8
        bit_idx = index % 8
        return byte_idx, bit_idx

    def _read_bit(self, index: int) -> int:
        """Lee el valor de un bit (0 o 1)."""
        b, i = self._bit_coords(index)
        return (self._bits[b] >> i) & 1

    def _write_bit(self, index: int) -> bool:
        """Establece un bit a 1. Devuelve True si cambió de estado."""
        b, i = self._bit_coords(index)
        mask = 1 << i
        old = self._bits[b]
        self._bits[b] |= mask
        return old != self._bits[b]

    def _key_positions(self, key: str):
        """Genera las posiciones de bit para una clave usando hashing doble."""
        s = consistent_stringify(key)
        h1 = murmurhash3_32(s, self._seed)
        h2 = fnv1_hash32(s)
        for i in range(self._num_hashes):
            # Combinación lineal de los hashes (double hashing + cuadrática)
            yield (h1 + i * h2 + i * i) % self._num_bits

    def add(self, value) -> "BloomFilter":
        """Añade un valor al filtro. Incrementa _size si algún bit cambió."""
        key = consistent_stringify(value)
        flipped = False
        for pos in self._key_positions(key):
            if self._write_bit(pos):
                flipped = True
        if flipped:
            self._size += 1
        return self

    def contains(self, value) -> bool:
        """Comprueba si un valor podría estar en el filtro (puede haber falsos positivos)."""
        key = consistent_stringify(value)
        return all(self._read_bit(pos) for pos in self._key_positions(key))

    @property
    def size(self) -> int:
        """Número de valores únicos insertados (aproximado)."""
        return self._size

    def false_positive_probability(self) -> float:
        """Calcula la probabilidad teórica de falso positivo."""
        k, n, m = self._num_hashes, self._size, self._num_bits
        return (1 - math.exp(-k * n / m))**k

    def confidence(self) -> float:
        """Devuelve la confianza (1 - probabilidad de falso positivo)."""
        return 1 - self.false_positive_probability()

    @property
    def max_remaining_capacity(self) -> int:
        """Capacidad restante antes de alcanzar max_size."""
        return max(0, self._max_size - self._size)


### **Pruebas unitarias**

Las **pruebas unitarias** permiten verificar que los componentes individuales de un programa (como clases o funciones) se comporten correctamente bajo distintos escenarios. En este caso, se usa el módulo `unittest` de Python para validar exhaustivamente la clase `BloomFilter`.

Cada método dentro de `TestBloomFilter(unittest.TestCase)` representa una prueba. Se utilizan aserciones como `assertTrue`, `assertEqual`, `assertRaises`, `assertGreater` y `assertLess` para comprobar el comportamiento esperado.

El código realiza lo siguiente:

1. **Prueba de interfaz (`test_interface`)**: Verifica que `BloomFilter` tenga los métodos públicos esperados (`add`, `contains`, etc.) y atributos (`size`, `max_remaining_capacity`).

2. **Validación de entradas inválidas (`test_invalid_max_size`, `test_invalid_tolerance`, `test_invalid_seed`)**: Se asegura de que el constructor rechace tipos incorrectos para `max_size`, `tolerance` y `seed`, lanzando `TypeError`.

3. **Errores por uso excesivo de memoria (`test_allocation_error`)**: Prueba que se lance un `MemoryError` si se intenta crear un filtro con parámetros poco realistas.

4. **Estado del filtro (`test_size_and_capacity`)**: Comprueba que `size` y `max_remaining_capacity` se actualicen correctamente, y que no aumente la cuenta al agregar duplicados.

5. **Precisión del filtro (`test_false_positive_and_confidence`)**: Evalúa el comportamiento del filtro conforme se agregan elementos y cambia la probabilidad de falsos positivos.

6. **Correctitud de `contains` (`test_contains_behavior`)**: Valida que un filtro vacío nunca contenga elementos, que se reconozcan los insertados, y que se produzcan falsos positivos solo con tolerancias altas.


In [ ]:
import unittest

class TestBloomFilter(unittest.TestCase):

    def test_interface(self):
        self.assertTrue(callable(BloomFilter))
        bf = BloomFilter(3)
        for method in ['contains', 'add', 'confidence', 'false_positive_probability']:
            self.assertTrue(hasattr(bf, method))
        for attr in ['size', 'max_remaining_capacity']:
            self.assertTrue(hasattr(bf, attr))

    def test_invalid_max_size(self):
        for invalid in ([], 'h', {'4':4}, None):
            with self.assertRaises(TypeError):
                BloomFilter(invalid)

    def test_invalid_tolerance(self):
        for tol in ([], 'g', {'1':2}, 0, -1, 1, 1.001):
            with self.assertRaises(TypeError):
                BloomFilter(3, tol)

    def test_invalid_seed(self):
        for seed in ([], 'g', {'1':2}, 0.1, 1.1):
            with self.assertRaises(TypeError):
                BloomFilter(3, 0.1, seed)

    def test_allocation_error(self):
        with self.assertRaises(MemoryError):
            BloomFilter(10**8, 1e-9)
        with self.assertRaises(MemoryError):
            BloomFilter(10**9, 1e-10)

    def test_size_and_capacity(self):
        maxS = 5
        bf = BloomFilter(maxS)
        self.assertEqual(bf.size, 0)
        self.assertEqual(bf.max_remaining_capacity, maxS)
        bf.add(12)
        self.assertEqual(bf.size, 1)
        bf.add(11); bf.add(13)
        self.assertEqual(bf.size, 3)
        self.assertEqual(bf.max_remaining_capacity, maxS - 3)
        bf.add(13); bf.add(11)
        self.assertEqual(bf.size, 3)
        for i in range(1, maxS+2):
            bf.add(i)
        self.assertEqual(bf.max_remaining_capacity, 0)

    def test_false_positive_and_confidence(self):
        maxS, maxT = 10, 0.1
        bf = BloomFilter(maxS, maxT)
        self.assertEqual(bf.false_positive_probability(), 0)
        bf.add('x')
        self.assertGreater(bf.false_positive_probability(), 0)
        while bf.size < maxS:
            bf.add(random.randint(0, maxS*2))
        self.assertGreater(bf.false_positive_probability(), maxT)
        bf2 = BloomFilter(15, 0.01)
        self.assertEqual(bf2.confidence(), 1)
        bf2.add('y')
        self.assertLess(bf2.confidence(), 1)

    def test_contains_behavior(self):
        maxS = 15
        bf = BloomFilter(maxS)
        self.assertTrue(all(not bf.contains(i) for i in range(maxS)))
        bf_low = BloomFilter(maxS, 1e-20)
        for i in range(maxS):
            bf_low.add(i)
        self.assertTrue(all(not bf_low.contains(maxS+i) for i in range(maxS*2)))
        bf_high = BloomFilter(maxS, 1e-1)
        for i in range(maxS):
            bf_high.add(i)
        self.assertTrue(any(bf_high.contains(maxS+i) for i in range(maxS*3)))
        bf3 = BloomFilter(maxS)
        for i in range(maxS):
            bf3.add(i)
        self.assertTrue(all(bf3.contains(i) for i in range(maxS)))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)


#### **Rendimiento y comportamiento interno de una estructura Bloom filter**

Utilizamos  dos herramientas comunes en Python:

- `%prun`: para **profiling puntual** de funciones individuales.
- `%timeit`: para **benchmark estadístico** de bloques de código repetidos.


Se inicializa un Bloom filter para hasta 5000 elementos con una tolerancia de falsos positivos del 1%. Luego se insertan 2000 valores para "calentar" el filtro y simular un uso realista.

`%prun` es una *magic command* de IPython que utiliza `cProfile` para mostrar estadísticas detalladas del tiempo de ejecución de cada función durante una llamada.

**Resultados**

```
Inserción de 2000 elementos:
19.3 ms ± 270 μs

Búsqueda de 2000 elementos existentes:
19.8 ms ± 307 μs

Búsqueda de 2000 elementos NO existentes:
14.9 ms ± 281 μs
```


**Análisis:**

| Operación | Tiempo Total (ms) | Tiempo Medio por elemento (μs) | Comentario |
|----------|------------------|-----------------------------|------------|
| `add(i)` | 19.3 ms           | ~9.65 μs                   | Muy eficiente |
| `contains(i)` en insertados | 19.8 ms           | ~9.9 μs                    | Ligeramente más costoso que insertar |
| `contains(i+10000)` no insertados | 14.9 ms | ~7.45 μs            | Más rápido: se detectan ceros antes |

- La búsqueda de **elementos no insertados** es más rápida porque en promedio encuentra bits en 0 más temprano, evitando completar las `k` verificaciones.
- Todos los tiempos están por debajo de los 10 microsegundos por operación, lo cual valida que Bloom Filter es **óptimo para pruebas de pertenencia en masa**.
- Las funciones `add` y `contains` realizan operaciones **muy eficientes**, con tiempos constantes.
- El perfil muestra que las operaciones clave (`hashing`, `bit setting`, `bit checking`) son rápidas y con mínimo *overhead* de serialización (`json.dumps`).
- El benchmark muestra tiempos competitivos incluso para grandes cantidades de operaciones, destacando la utilidad de Bloom filters en sistemas de alto rendimiento como bases de datos, motores de búsqueda o sistemas distribuidos.

In [ ]:
# Prepara un filtro y caliéntalo con unas inserciones
bf = BloomFilter(5000, 0.01, seed=42)
for i in range(2000):
    bf.add(i)

# Perfilado de operaciones puntuales (método add y contains)
print("Perfilado con %prun (top 10)")
print("bf.add(1234):")
%prun -l 10 bf.add(1234)

print("\nbf.contains(1234):")
%prun -l 10 bf.contains(1234)

# Benchmark de rendimiento con %timeit
print("\nBenchmark con %timeit ")
print("Inserción de 2000 elementos:")
%timeit -n5 -r3 [bf.add(i) for i in range(2000)]

print("\nBúsqueda de 2000 elementos existentes:")
%timeit -n5 -r3 [bf.contains(i) for i in range(2000)]

print("\nBúsqueda de 2000 elementos NO existentes:")
%timeit -n5 -r3 [bf.contains(i+10000) for i in range(2000)]


### **Estadísticas y gráficos adicionales**

Este código realiza un análisis estadístico y visual del rendimiento de tres operaciones en un **Bloom Filter**:

1. `bf.add(1234)`
2. `bf.contains(1234)` (elemento insertado)
3. `bf.contains(10000)` (elemento no insertado)

Se ejecutan **30 veces** cada una con `timeit.repeat()` y se almacenan los tiempos en segundos. Luego, se calcula la **media**, **mediana** y **desviación estándar** usando el módulo `statistics`.

Los resultados se organizan en un `DataFrame` de **pandas**, que se muestra como tabla. Y produce los siguiente gráficos:

- **Gráfico de barras**: compara los **tiempos medios** por operación. Ayuda a identificar cuál es más costosa.
- **Histograma**: muestra la **distribución de los tiempos** para `bf.add()`, revelando la variabilidad o estabilidad del rendimiento.

Este análisis permite observar que:

- `contains(10000)` suele ser más rápido (detecta ausencia temprana).
- `add` y `contains` para elementos existentes tienen mayor consistencia pero tiempos similares.
- El uso de estadísticas robustas como mediana y desviación da contexto sobre la **estabilidad temporal** de cada operación.

In [ ]:
import timeit
import statistics
import matplotlib.pyplot as plt

# Prepara de nuevo el filtro (o reutiliza el 'bf' de antes si sigue en memoria)
bf = BloomFilter(5000, 0.01, seed=42)
for i in range(2000):
    bf.add(i)

# Número de muestras para cada operación
repeats = 30

# Recopila los tiempos como listas de floats (segundos)
times_add = timeit.repeat(lambda: bf.add(1234), repeat=repeats, number=1)
times_contains_exist = timeit.repeat(lambda: bf.contains(1234), repeat=repeats, number=1)
times_contains_not = timeit.repeat(lambda: bf.contains(10000), repeat=repeats, number=1)

# Calcula media, mediana y desviación típica
stats = {
    'operación': ['add', 'contains_exist', 'contains_not'],
    'media (s)': [
        statistics.mean(times_add),
        statistics.mean(times_contains_exist),
        statistics.mean(times_contains_not)
    ],
    'mediana (s)': [
        statistics.median(times_add),
        statistics.median(times_contains_exist),
        statistics.median(times_contains_not)
    ],
    'desviación (s)': [
        statistics.stdev(times_add),
        statistics.stdev(times_contains_exist),
        statistics.stdev(times_contains_not)
    ]
}

# Muestra la tabla de estadísticas
import pandas as pd
df = pd.DataFrame(stats)
display(df)

# — Gráfico 1: Barras del tiempo medio por operación —
plt.figure()
plt.bar(df['operación'], df['media (s)'])
plt.ylabel('Tiempo medio (s)')
plt.title('Tiempo medio por operación')
plt.show()

# — Gráfico 2: Histograma de distribución de bf.add() —
plt.figure()
plt.hist(times_add, bins=10)
plt.xlabel('Tiempo de ejecución (s)')
plt.title('Distribución de tiempos de bf.add()')
plt.show()


### **Probabilidad de falso positivo y uso de memoria**

Este código realiza un **análisis empírico y teórico del comportamiento de un Bloom filter**, evaluando dos aspectos fundamentales:

1. La **probabilidad de falso positivo** (teórica vs. empírica).
2. El **uso de memoria** en función del número de elementos insertados.


Se define una serie de tamaños de carga (`sizes`) desde 0 hasta 5000 elementos, en pasos de 500. Para cada carga se mide:

- **FP teórica**: basada en la fórmula matemática del filtro.
- **FP empírica**: midiendo cuántos valores *no insertados* son reconocidos erróneamente como presentes.
- **Uso de memoria**: en bytes del `bytearray` del filtro.


Para cada valor de `n` en `sizes`:

1. Se crea un nuevo filtro con capacidad máxima de 5000 y tolerancia de error de 0.01.
2. Se insertan `n` elementos únicos (`0, 1, ..., n-1`).
3. Se calcula:
   - `bf.false_positive_probability()`: estimación teórica.
   - `fp_count / test_samples`: estimación empírica sobre `test_samples` enteros aleatorios que no han sido insertados.
   - `sys.getsizeof(bf._bits)`: memoria usada por el arreglo de bits.



In [ ]:
import random
import sys
import matplotlib.pyplot as plt

# Parámetros de barrido
max_inserts = 5000
step = 500
test_samples = 2000  # cuántos valores no insertados probamos para FP
sizes = list(range(0, max_inserts + 1, step))

# Listas para almacenar resultados
fp_theoretical = []
fp_empirical  = []
mem_usage_bytes = []

for n in sizes:
    # Crea y rellena el filtro con n elementos
    bf = BloomFilter(5000, 0.01, seed=42)
    for i in range(n):
        bf.add(i)
    
    # 1) Probabilidad teórica de FP
    fp_theoretical.append(bf.false_positive_probability())
    
    # 2) Medición empírica de FP
    fp_count = 0
    for _ in range(test_samples):
        candidate = random.randint(max_inserts*2, max_inserts*3)
        if bf.contains(candidate):
            fp_count += 1
    fp_empirical.append(fp_count / test_samples)
    
    # 3) Memoria usada por el array de bits
    mem_usage_bytes.append(sys.getsizeof(bf._bits))

# Gráfica 1: FP teórica vs empírica
plt.figure()
plt.plot(sizes, fp_theoretical, label='Teórica')
plt.plot(sizes, fp_empirical,  label='Empírica', linestyle='--')
plt.xlabel('Número de elementos insertados')
plt.ylabel('Probabilidad de falso positivo')
plt.title('FP teórica vs FP empírica')
plt.legend()
plt.show()

# Gráfica 2: Uso de memoria del bitarray
plt.figure()
plt.plot(sizes, mem_usage_bytes)
plt.xlabel('Número de elementos insertados')
plt.ylabel('Bytes usados por el bitarray')
plt.title('Consumo de memoria según cargas')
plt.show()


La primera gráfica compara la probabilidad **teórica** de falsos positivos frente a la **medida empíricamente**, a medida que se insertan más elementos.

- La curva **teórica** (azul sólida) sigue la fórmula:
  $$
  P_{fp} = \left(1 - e^{-kn/m}\right)^k
  $$
  donde $n$ = número de elementos insertados.
- La curva **empírica** (naranja discontinua) se basa en la proporción real de falsos positivos.
- Ambas curvas coinciden en general, validando el modelo teórico.
- Hacia el final (más carga), la curva empírica tiende a superar levemente la teórica, lo cual es esperado por el efecto de colisiones y límite de capacidad.


La segunda gráfica muestra el uso de memoria del arreglo de bits del filtro (`bf._bits`), medido en bytes.

**Observaciones:**

- La línea es **horizontal**, constante en todos los puntos.
- Esto refleja una propiedad clave de los Bloom filters: **el tamaño del arreglo de bits no crece con el número de elementos insertados**, sino que se fija en el momento de la construcción del filtro.
- En este caso, el arreglo tiene aproximadamente 6031 bytes desde el inicio.
- El comportamiento no cambia aunque el filtro esté vacío o completamente lleno.

Este experimento ilustra claramente dos propiedades fundamentales de los Bloom filters:

- **Eficiencia de espacio**: uso de memoria constante y predecible, lo que los hace ideales para aplicaciones con restricciones de memoria.
- **Naturaleza probabilística**: los falsos positivos crecen de manera predecible conforme se llena el filtro. La concordancia entre teoría y observación empírica valida su formulación matemática.

Este tipo de visualización es crucial para comprender cuándo un Bloom filter comienza a ser menos confiable y para ajustar sus parámetros de diseño (capacidad máxima, tolerancia al error) en sistemas reales.

### Ejercicios

1. **Análisis de complejidad y eficiencia**  
   Justifica matemáticamente por qué un Bloom filter puede considerarse una estructura de datos con tiempo constante $ O(k) $ tanto en inserción como en consulta, e identifica los límites de esta afirmación bajo implementaciones reales (por ejemplo, en entornos con restricciones de memoria caché o compresión).

2. **Optimización del número de funciones hash**  
   Demuestra a partir de la fórmula de falsos positivos que existe un valor óptimo de $ k $, el número de funciones hash, y deduce la expresión $ k = \frac{m}{n} \ln 2 $. ¿Qué implicaría usar más o menos funciones hash que este valor óptimo?

3. **Diseño de variantes del Bloom filter**  
   Explica cómo se podría diseñar un Counting Bloom Filter para permitir eliminaciones, y analiza su sobrecosto de memoria. Luego, describe un escenario donde esta variante es indispensable.

4. **Evaluación del error empírico**  
   Suponiendo que insertas 4000 elementos en un Bloom filter con $ m = 48000 $ bits y $ k = 7 $, estima la probabilidad empírica de falso positivo. Luego, diseña un experimento que pueda validar esta estimación con un margen de error del 1%.

5. **Hashing y colisiones**  
   Describe cómo el uso de funciones hash con poca dispersión puede afectar gravemente la tasa de falsos positivos en Bloom filters. ¿Qué propiedades debe tener una función hash para minimizar este problema?

6. **Diseño para sistemas distribuidos**  
   Analiza cómo se pueden usar Bloom filters en arquitecturas distribuidas para reducir latencia y tráfico de red. Menciona al menos tres casos reales donde esto tenga impacto en bases de datos o redes.

7. **Límites teóricos y físicos**  
   Si quisiéramos reducir la tasa de falsos positivos a $10^{-6}$ para almacenar 1 millón de elementos, ¿cuánto espacio en bits se requeriría? ¿Sería práctico usar este filtro en una aplicación en memoria en un sistema embebido?

8. **Construcción dinámica**  
   Implementa un Scalable Bloom Filter que se expanda automáticamente al superar la capacidad prevista, generando nuevas capas de filtros con tasas de error progresivamente más pequeñas.

9. **Medición empírica**  
   Diseña un script que compare la tasa de falsos positivos en distintos escenarios (diferentes tamaños de filtro y valores de $ k $) y genera gráficos automáticos para comparar curvas teóricas vs empíricas.

10. **Carga y serialización eficiente**  
   Implementa funciones para exportar e importar Bloom filter a disco en formato comprimido y serializado, asegurando que el filtro pueda ser restaurado exactamente con su semilla y estado.

11. **Análisis de sensibilidad**  
   Programa un experimento que mida cómo varía el rendimiento (tiempo de inserción y consulta) cuando se cambia el número de funciones hash $ k $, manteniendo $ m $ y $ n $ fijos.

12. **Validación cruzada**  
   Crea un conjunto de pruebas unitarias robustas que verifiquen que el filtro:
       - No presenta falsos negativos.
       - Presenta una tasa de falsos positivos dentro del margen teórico esperado.
       - Se comporta igual con elementos serializados en diferente orden (test de `consistent_stringify`).

13. **Interfaz tipo set probabilístico**  
   Diseña una clase que envuelva un Bloom filter con una API similar a la clase `set` de Python (`add`, `__contains__`, `len`), y que devuelva advertencias si la probabilidad de falso positivo excede cierto umbral.

14. **Compresión del bitarray**  
   Implementa una técnica de compresión del `bitarray` (como RLE o gzip) para enviar Bloom filters entre nodos en una red, y evalúa cuánto se reduce el tamaño sin perder precisión en falsos positivos.


In [ ]:
### Tus respuestas